# Lab G: Exploration: LIDAR

このノートでは、RaceacrのLiDARを使って距離を測定し、最も近い物体をを見つける方法を学びます。

このノートブック全体を通して **<font style="color:red">太い赤字で書かれた文章</font>** は、実行する前にその下のコードブロックを編集して正しいコードを書く必要があります。


## 目次
1. [はじめに](#GettingStarted)
1. [LiDARデータの収集](#GatheringLidarData)
1. [LiDARデータの可視化](#VisualizingLidarData)
1. [ノイズの扱い方](#HandlingNoise)
1. [最も近い点](#ClosestPoint)

<a id="GettingStarted"></a>
## 1. はじめに

**<font style="color:red">もしシミュレータを利用して開発を進める場合は、 `isSimulation` を `True` に設定します </font>**。 実際のマシンを利用する場合は、 `isSimulation` を `False` のままにしてください。

In [ ]:
# TODO: 必要に応じてisSimulationを更新する
isSimulation = True

次に、Pythonライブラリ(`cv`, `numpy`, など)や、Racecarライブラリ(`racecar_core`)など、このノートブックの実行に必要なライブラリをインポートします。

In [ ]:
# Pythonライブラリのインポート
import math
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
import statistics
from nptyping import NDArray
from typing import Any, Tuple, List, Optional

# Racecarライブラリのインポート
import sys
sys.path.append("../../library")
import racecar_core
import racecar_utils as rc_utils

最後に、Racecarオブジェクトを作成します。このステップで失敗した場合は`isSimulation` が正しい値であることを確認してください。

In [ ]:
# Racecar オブジェクトの作成
rc = racecar_core.create_racecar(isSimulation)

<a id="GatheringLidarData"></a>
## 2. LiDARデータの収集
RacecarのLiDARデータは、720個の距離測定値を1次元numpy配列として格納しています。 各測定値は1/2度（＝0.5°）間隔で、時計回りに並んでおり、0番目のデータは車の真正面にあります。

Jupyter Notebookでは、`rc.lidar.get_samples_async()`を使って車のLiDARデータにアクセスすることができます。 Jupyter Notebookの外では、代わりに`rc.lidar.get_samples()`を使用する必要があります。

In [ ]:
# 現在のLiDARのデータを取得する
scan = rc.lidar.get_samples_async()

早速、Racecarの真正面と真後ろの距離を測定してみましょう！

In [ ]:
# 前方と後方の距離を表示する
forward_distance = scan[0]
print(f"Forward distance: {forward_distance:.2f} cm")

rear_distance = scan[360]
print(f"Rear distance: {rear_distance:.2f} cm")

**<span style="color:red">以下のコードブロックの`left_distance`と`right_distance`に、Racecarの左側と右側のLiDAR測定値に設定してみましょう。</span>**

In [ ]:
# TODO: 左側と右側の距離を表示する
left_distance = scan[0]
print(f"Left distance: {left_distance:.2f} cm")

right_distance = scan[0]
print(f"Right distance: {right_distance:.2f} cm")

<a id="VisualizingLidarData"></a>
## 3. LiDARデータの可視化

このセクションでは、LiDARデータをカラー画像に変換する関数を作成します。  
以下のアプローチを使用します：

1. 指定した半径の真っ黒な正方形のBGR画像（行、列、色チャンネルを並べた3D numpy配列）を作成する。
1. [`rc_utils.draw_circle`](https://mitll-racecar.readthedocs.io/en/latest/racecar_utils.html#racecar_utils.draw_circle)を使って、画像の中心に緑色の点を描く。これはRacecarの位置を示します。
1. 各LiDARデータについて、対応するピクセルを赤に設定します。取得したデータのインデックスから各サンプルの角度を計算します。 また、`max_range`が画像の端になるように距離をスケーリング（変換）することで、この角度と距離を画像の行と列に変換することができます。
1. `highlighted_samples` には、水色の点で強調したい `(角度, 距離)` 測定値のリストが格納されます。これらは前のステップのサンプルと同様にプロットし、`rc_utils.draw_circle`で描画することができます。
1. Matplotlib を使ってJupyter Notebookにカラー画像を表示します。

結果は、RacecarSimの画面左側にに表示されているLiDARを可視化した画像と同じように見えるはずです。

**<span style="color:red">以下の `show_lidar` 関数を上記のアプローチで実装してみましょう。</span>**

In [ ]:
def show_lidar(
    scan: NDArray[Any, np.float32],
    radius: int = 128,
    max_range: int = 400,
    highlighted_samples: List[Tuple[int, int]] = []
) -> None:
    """
    Jupyter NotebookでLiDARデータを視覚的に表示します
    
    Args:
        scan: 表示するLiDARデータ
        radius: 生成された画像の幅と高さの半分の値（ピクセル単位）
        max_range: 画像に表示する最も遠い距離（cm）。この範囲を超えたサンプルは表示されない。
        highlighted_samples: (角度、距離) フォーマットの強調表示するデータのリスト
    """    
    # 要求された半径で正方形の黒い画像を作成する
    image = np.zeros((2 * radius, 2 * radius, 3), np.uint8, "C")
    num_samples: int = len(scan)

    # TODO: 画像の中心にRacecarを示す緑色の点を表示する
    # ヒント: rc_utils.draw_circle を使いましょう
    CAR_DOT_RADIUS = 2

        
    # TODO: max_rangeより小さいゼロではないデータごとに赤いピクセルを描画する

    
    # TODO: highlighted_samplesで指定された位置にそれぞれ水色の点を描画する
    # ヒント: rc_utils.draw_circle を使いましょう
    HIGHLIGHT_DOT_RADIUS = 2


    # Matplotlibで画像を表示する
    plt.imshow(cv.cvtColor(image, cv.COLOR_BGR2RGB))
    plt.show()

それでは、この関数を使ってLiDARデータを視覚化してみよう。

In [ ]:
show_lidar(scan)

とりあえず、各方向に100cmの点を強調表示して、`highlighted_samples`機能を試してみましょう。

In [ ]:
show_lidar(scan, highlighted_samples=[(0, 100), (90, 100), (180, 100), (270, 100)])

<a id="HandlingNoise"></a>
## 4. ノイズの扱い方

LiDARデータはノイズやNULL値（無効な値）の影響を受けます。これに対処するために、1つの測定値に頼るのではなく、*角度ウィンドウ*をまたいで隣接する複数のサンプルを平均化する。ここで*角度ウィンドウ*とは、ある特定の角度周りにある一定範囲の角度データをまとめることを意味します。例えば、Racecarから見て、60度の方向の距離データを測定したいとき、角度ウィンドウの幅を4度に設定すると、60度の前後2度、つまり58度から62度までのデータを集めて、その平均値を取ることになります。  
また、測定値の中に無効な値（ここでは「0.0」となっているもの）が含まれていると平均値に悪影響を与えてしまうため、これらの0.0の値は平均の計算空除外します。

**<span style="color:red">それでは、このアプローチを`get_lidar_average_distance`で実装してみましょう。</span>** Pythonの [内包表記](https://docs.python.org/ja/3.13/tutorial/datastructures.html#list-comprehensions)を利用すると良いでしょう。

In [ ]:
def get_lidar_average_distance(
    scan: NDArray[Any, np.float32], angle: float, window_angle: float = 4
) -> float:
    """
    Racecarに対して特定の角度にある物体の平均距離を求める

    Args:
        scan: LiDARのデータ
        angle: 距離を測定する角度(degree)。
            Racecarの前方を0度として、時計周りに増加することに注意。
        window_angle: 角度の周辺を考慮する度数

    Returns:
        指定した角度の点の平均距離 (cm)

    Note:
        値が0.0 (データなし)のサンプルは無視する
        window_angleを大きくすると、精度が落ちる代わりにノイズの影響を受けにくくなります
    """
    # TODO: 指定した角度ウィンドウ内のサンプルを平均化する


それでは試してみましょう！  
角度ウィンドウの角度を大きくすると、より多くのサンプルが含まれるためノイズに強くなりますが、関連性の低いサンプルを含むようになるため精度が低下してしまいます。

In [ ]:
WINDOW_ANGLE = 6
rear_distance = get_lidar_average_distance(scan, 180, WINDOW_ANGLE)
print(f"Rear distance ({WINDOW_ANGLE} degree window): {rear_distance:.2f} cm")

`get_lidar_average_distance` は角度ウィンドウが配列の端をまたぐ場合を考慮する必要があります。例えば、指定した角度が0度で角度ウィンドウの値が6の場合、0から前後に3度の値つまり357度から3度の範囲のデータで平均を計算する必要があります。

**<span style="color:red">もし、上記のような処理を`get_lidar_average_distance`に実装していなければ、配列の端をまたぐ角度ウィンドウでも問題なく動くように追記修正しましょう。</span>**

In [ ]:
forward_distance = get_lidar_average_distance(scan, 0, WINDOW_ANGLE)
print(f"Forward distance ({WINDOW_ANGLE} degree window): {forward_distance:.2f} cm")

最後に、指定した角度ウィンドウにデータがない場合の処理を追加する必要があります。この場合は`0.0`を返すようにしましょう。

**<span style="color:red">もし、上記のような処理を`get_lidar_average_distance`に実装していなければ、指定した範囲にデータがない場合は`0.0`を返すように追記修正してください。</span>**

In [ ]:
null_scan = np.zeros(rc.lidar.get_num_samples(), np.float32)
forward_distance = get_lidar_average_distance(null_scan, 0, WINDOW_ANGLE)
print(f"Forward distance ({WINDOW_ANGLE} degree window) in null scan: {forward_distance:.2f} cm")

<a id="ClosestPoint"></a>
## 5. 最も近い点

LiDARを使って、Raceacrの周り360度にある物体のうち、一番近くにあるものの角度を見つけることができます。
もし測定できなかった（無効な）値があった場合、その値を極端に大きな値に変換して、最小値の計算に入らないようにします。
そのための効率的な方法の一つが、各値を少しだけ減らして（たとえば0.01 cmだけ）、その後に大きな数（たとえば10,000 cm）で割った余り（mod）を求める方法です。
この操作を行うと、例えば0.0はまず-0.01になりますが、その後にmodを計算すると9,999.99 cmとなります。
このようにして、無効な値は実際の距離計測値とは大きく異なる値になり、一番近い物体の距離を正しく求めることができるのです。

```
scan = (scan - 0.01) % 10000
```

例えば、Numpyの[argmin](https://numpy.org/doc/1.19/reference/generated/numpy.argmin.html)関数を利用して、最も近い点が度の角度にあるかを求める事ができます。

```
scan = (scan - 0.01) % 10000
angle = np.argmin(scan) * 360 / rc.lidar.get_num_samples()
```

しかし、場合によっては、30度から150度までのように、特定の範囲内のサンプルだけを対象にしたいこともあります。
**<span style="color:red">`get_closest_pixel`を実装して、LiDARデータからして下ウィンドウ内の最も近い点の角度を距離を求めてみましょう。</span>**.

In [ ]:
def get_lidar_closest_point(
    scan: NDArray[Any, np.float32], window: Tuple[float, float] = (0, 360)
) -> Tuple[float, float]:
    """
    LiDARのデータから最も近い点を見つける
    Args:
        scan: LiDARのデータ
        window: (min_degree, max_degree)で表される考慮する角度の範囲。

    Returns:
        指定された角度ウィンドウ内で、最もRaceacarに近い点 (角度、距離)を返す
        角度の単位は度(degree)で、Raceacarの真正面が0度で、時計回りに増加する
    Note:
        値が0.0(データなし)のデータは無視される

        360度～0度の教会を通過する角度ウィンドウを利用する場合には、以下のようにする。
        window min_degreeはwindow max_degreeよりも大きくてもよい。
        例えば、(350, 10)はRaceacarの前方20度の角度ウィンドウを指定したことになる。
    """
    # TODO: 指定したウィンドウ内で最も近い点の(角度、距離）を戻す
    

それでは、`get_lidar_closest_point`を使って、Racecarの右側でもっとも近い点を探してみましょう。

In [ ]:
angle, distance = get_lidar_closest_point(scan, (30, 150))
print(f"Angle: {angle:.1f} degrees")
print(f"Distance: {distance:.1f} cm")

show_lidar(scan, highlighted_samples=[(angle, distance)])

繰り返しになりますが、指定された角度ウィンドウが配列の端をまたぐ場合にも対処する必要があります。  **<span style="color:red">まだ`get_lidar_closest_point`に機能を実装していない場合には、負の角度と配列の端をまたぐ角度ウィンドウをサポートするように追記修正しましょう。</span>**

In [ ]:
angle, distance = get_lidar_closest_point(scan, (-30, 30))
print(f"Angle: {angle:.1f} degrees")
print(f"Distance: {distance:.1f} cm")

show_lidar(scan, highlighted_samples=[(angle, distance)])

これで、LiDARのデータを扱う`lab_g1.py`と`lab_g2.py`に取り組む準備ができました。 幸運を祈ります！ 何か質問があれば、遠慮なく声をかけてください！